In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import xarray as xr
import glob
import os.path
import datetime
import gc

new_names = ["Longitude","Latitude","DatetimeBegin","SSR"]
st = pq.read_table("../AQ_stations/EEA_stations_meta.parquet").to_pandas()
countries = st.Countrycode.unique()

#countries = ['LU', 'AD']

ssr_file = glob.glob(f"../supplementary/era5/*solar*")
ssr = xr.open_mfdataset(ssr_file)
ssr

<xarray.Dataset>
Dimensions:    (longitude: 663, latitude: 460, time: 70128)
Coordinates:
  * longitude  (longitude) float32 -30.63 -30.53 -30.43 ... 35.37 35.47 35.57
  * latitude   (latitude) float32 71.66 71.56 71.46 71.36 ... 25.96 25.86 25.76
  * time       (time) datetime64[ns] 2015-01-01 ... 2022-12-31T23:00:00
Data variables:
    ssr        (time, latitude, longitude) float32 dask.array<chunksize=(8760, 460, 663), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-10-08 00:41:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [18]:
%%time

for c in countries:
    aq = pq.read_table(f"../AQ_data/01_hourly_gaps/{c}_hourly_2015-2023_gaps.parquet").to_pandas()
    out_pq = f"../AQ_data/02_hourly_SSR/{c}_hourly_gaps_SSR.parquet"
    print(f"***** {c} *****")

    if not os.path.isfile(out_pq):

        # define extract locations in 3D
        target_lon = xr.DataArray(aq["Longitude"].values, dims="points")
        target_lat = xr.DataArray(aq["Latitude"].values, dims="points")
        target_time = xr.DataArray(aq["DatetimeBegin"].values.astype("datetime64[ns]"), dims="points")

        # extract
        ssr_ex = ssr.ssr.sel(longitude = target_lon,
                         latitude = target_lat, 
                         time = target_time,
                         method = "nearest")
        ssr_df = ssr_ex.to_series().rename("SSR")

        result = aq.merge(ssr_df, left_index=True, right_index=True)
        result.to_parquet(out_pq)
        ssr.close()
        gc.collect()
        print(datetime.datetime.now() )

***** AD *****
***** AL *****
***** AT *****
***** BA *****
***** BE *****
***** BG *****
***** CH *****
***** CY *****
***** CZ *****
***** DE *****
***** DK *****
***** SE *****
***** EE *****
***** ES *****
***** FI *****
***** FR *****
***** GB *****
***** GI *****
***** GR *****
***** HR *****
***** HU *****
***** IE *****
***** IS *****
***** IT *****
***** LT *****
***** LU *****
***** LV *****
***** ME *****
***** MK *****
***** MT *****
***** NL *****
***** NO *****
***** PL *****
***** PT *****
***** TR *****
***** RO *****
***** RS *****
***** SI *****
***** SK *****
***** XK *****
CPU times: user 2h 23min 21s, sys: 2h 57min 21s, total: 5h 20min 43s
Wall time: 10h 22min 33s
